In [ ]:
# Import
import sqlite3
import pandas.io.sql as pds
import pandas as pd
from datetime import datetime, date
from ipywidgets import Layout, widgets, Style, Button
from IPython.display import display, clear_output

In [ ]:
!gdown 1Oprvp6bkFkGi-XO5Q2ug3SCczg8vh13U

Downloading...
From: https://drive.google.com/uc?id=1Oprvp6bkFkGi-XO5Q2ug3SCczg8vh13U
To: /content/Database Final Project.db
100% 270k/270k [00:00<00:00, 85.8MB/s]


# GUI

In [ ]:
# Tampilan Awal
email_widget = widgets.Text(
    value='',
    description='Email:',
    style={'description_width': 'initial'})

password_widget = widgets.Password(
    value='',
    description='Password:',
    style={'description_width': 'initial'})

login_button = widgets.Button(
    description='Login',
    button_style='success',
    tooltip='Login to your account')

register_button = widgets.Button(
    description='Register',
    button_style='info',
    tooltip='Create a new account')

output_widget = widgets.Output()

conn = None
cursor = None

# Koneksi ke Database
def connect_to_db():
    global conn, cursor
    conn = sqlite3.connect('Database Final Project.db')
    cursor = conn.cursor()

# Memeriksa apakah email sudah pernah digunakan sebelumnya
def enter(email, password):
    query = "SELECT * FROM account WHERE email =?"
    cursor.execute(query, (email,))
    result = cursor.fetchone()
    if result:
        with output_widget:
            clear_output()
            print("You must enter an email that you have never used")
        return False
    else:
        query = "INSERT INTO account (email, password) VALUES (?,?)"
        cursor.execute(query, (email, password))
        conn.commit()
        query = "SELECT last_insert_rowid()"
        cursor.execute(query)
        account_id = cursor.fetchone()[0]  # Mengambil account_id
        with output_widget:
            clear_output()
        return account_id  # Return account_id

# Mencocokkan input dengan data email di tabel account
def check(email, password):
    query = 'SELECT * FROM account WHERE email =? AND password =?'
    cursor.execute(query, (email, password))
    result = cursor.fetchone()
    conn.commit()
    if result:
        global account_id
        account_id = result[0]  # Menyimpan account_id
        return True
    else:
        return False

# Login
def login(b):
    with output_widget:
        clear_output()
        email = email_widget.value
        password = password_widget.value
        if check(email, password):
            query_jobseeker = "SELECT 1 FROM jobseeker WHERE account_id =?"
            query_company = "SELECT 1 FROM company WHERE account_id =?"
            cursor.execute(query_jobseeker, (account_id,))
            result_jobseeker = cursor.fetchone()
            cursor.execute(query_company, (account_id,))
            result_company = cursor.fetchone()
            if result_jobseeker:
                display_menu_jobseeker()
            elif result_company:
                display_menu_company()
            else:
                print("Account not found in either jobseeker or company table")
        else:
            print("Something wrong")

# Register akun baru
def register(b):
    with output_widget:
        clear_output()
        email = email_widget.value
        password = password_widget.value
        account_id = enter(email, password)  # Mengambil account_id dari fungsi enter
        if not account_id:
            print("Failed")
        else:
            print("Success! Please choose:")
            display_register_menu(account_id)

def display_register_menu(account_id):
    login_menu_widget = widgets.VBox([
        widgets.Button(description='Jobseeker', button_style='info', tooltip='Login as Jobseeker'),
        widgets.Button(description='Company', button_style='info', tooltip='Login as Company')
    ])
    display(login_menu_widget)
    jobseeker_button = login_menu_widget.children[0]
    jobseeker_button.on_click(lambda b: register_jobseeker(account_id))
    company_button = login_menu_widget.children[1]
    company_button.on_click(lambda b: register_company(account_id))

# Register akun sebagai Company
def register_company(account_id):
    clear_output()
    company_name_widget = widgets.Text(value='', description='Company Name:', style={'description_width': 'initial'})
    location_widget = widgets.Text(value='', description='Location:', style={'description_width': 'initial'})
    display(company_name_widget, location_widget)
    register_company_button = widgets.Button(description='Register Company')
    register_company_button.on_click(lambda b: complete_register_company(company_name_widget.value, location_widget.value, account_id))
    display(register_company_button)

def complete_register_company(company_name, location, account_id):
    query = "INSERT INTO company (name_com, location, account_id) VALUES (?,?,?)"
    cursor.execute(query, (company_name, location, account_id))
    conn.commit()
    print("Company registration complete!")
    login_again_button = widgets.Button(description='Login Again')
    login_again_button.on_click(lambda b: login_again())
    display(login_again_button)

# Register akun sebagai Jobseeker
def register_jobseeker(account_id):
    clear_output()
    name_widget = widgets.Text(value='', description='Name:', style={'description_width': 'initial'})
    address_widget = widgets.Textarea(value='', description='Address:', style={'description_width': 'initial'})
    phone_widget = widgets.Text(value='', description='Phone:', style={'description_width': 'initial'})
    education_widget = widgets.Dropdown(options=['High School','Bachelor\'s Degree', 'Master\'s Degree', 'PhD'],
                                        value=None,
                                        description='Education:',
                                        style={'description_width': 'initial'})
    birth_date = widgets.DatePicker(description='Birth Datw:', style={'description_width': 'initial'})
    display(name_widget, address_widget, phone_widget, education_widget, birth_date)
    register_jobseeker_button = widgets.Button(description='Register Jobseeker')
    register_jobseeker_button.on_click(lambda b: complete_register_jobseeker(
        name_widget.value,
        address_widget.value,
        phone_widget.value,
        education_widget.value,
        birth_date.value.strftime('%d/%m/%Y'),
        account_id
    ))
    display(register_jobseeker_button)

def complete_register_jobseeker(name, address, phone, education, birth_date, account_id):
    query = "INSERT INTO jobseeker (name_js, address, phone_number, education, birth_date, account_id) VALUES (?,?,?,?,?,?)"
    cursor.execute(query, (name, address, phone, education, birth_date, account_id))
    conn.commit()
    print("Jobseeker registration complete!")
    login_again_button = widgets.Button(description='Login Again')
    login_again_button.on_click(lambda b: login_again())
    display(login_again_button)

# Login ulang setelah Register
def login_again():
    clear_output()
    with output_widget:
        clear_output()
        display(email_widget)
        display(password_widget)
        display(login_button)
    display(output_widget)

## Menu Jobseeker
def display_menu_jobseeker():
    clear_output()
    print("Main Menu")
    main_menu_widget = widgets.VBox([
        widgets.Button(description='Profile', button_style='info', tooltip='View your profile'),
        widgets.Button(description='Search Job', button_style='info', tooltip='Search for jobs'),
        widgets.Button(description='Result', button_style='info', tooltip='View results'),
        widgets.Button(description='Interview', button_style='info', tooltip='View interview schedule'),
        widgets.Button(description='Apply Job', button_style='info', tooltip='apply job'),
        widgets.Button(description='Update Resume', button_style='info', tooltip='Upload or update your resume')
    ])
    display(main_menu_widget)
    jobseeker_info_button = main_menu_widget.children[0]
    jobseeker_info_button.on_click(lambda b: display_jobseeker_info(account_id))
    search_job_button = main_menu_widget.children[1]
    search_job_button.on_click(lambda b: search_job(b))
    view_results_button = main_menu_widget.children[2]
    view_results_button.on_click(lambda b: display_results(b))
    view_interview_button = main_menu_widget.children[3]
    view_interview_button.on_click(lambda b: display_interview(b))
    apply_job_button = main_menu_widget.children[4]
    apply_job_button.on_click(lambda b: apply_job(b))
    update_resume_button = main_menu_widget.children[5]
    update_resume_button.on_click(lambda b: update_resume(b))

# Mencari pekerjaan
def search_job(b):
    clear_output()
    job_title_input = widgets.Text(value='', description='Job Title:', style={'description_width': 'initial'})
    search_output = widgets.Output()
    display(job_title_input, search_output)
    def search_job_callback(b):
        job_title = job_title_input.value
        query = """
            SELECT j.job_id, j.title, j.date, j.type, j.description, j.deadline, j.salary, c.name_com, q.qualification, r.requirement
            FROM job j
            LEFT JOIN company c ON j.company_id = c.company_id
            LEFT JOIN br_qualif bq ON j.job_id = bq.job_id
            LEFT JOIN qualif q ON bq.Qualif_id = q.qualif_id
            LEFT JOIN br_requir br ON j.job_id = br.job_id
            LEFT JOIN requir r ON br.req_id = r.req_id
            WHERE j.title LIKE '%' || ? || '%'
        """
        cursor.execute(query, (job_title,))
        results = cursor.fetchall()

        if results:
            data = {}
            for row in results:
                job_id = row[0]
                if job_id not in data:
                    data[job_id] = {
                        'title': row[1],
                        'date': row[2],
                        'type': row[3],
                        'description': row[4],
                        'deadline': row[5],
                        'salary': row[6],
                        'name_com': row[7],
                        'qualifications': set(),
                        'requirements': set()
                    }
                if row[8]:  # Menambahkan qualifications jika ada
                    data[job_id]['qualifications'].add(row[8])
                if row[9]:  # Menambahkan requirements jika ada
                    data[job_id]['requirements'].add(row[9])

            # Mengubah processed data menjadi dataframe
            processed_results = []
            for job_id, job_info in data.items():
                processed_results.append([
                    job_id,
                    job_info['title'], job_info['date'], job_info['type'], job_info['description'],
                    job_info['deadline'], job_info['salary'], job_info['name_com'],
                    ', '.join(job_info['qualifications']), ', '.join(job_info['requirements'])
                ])

            df = pd.DataFrame(processed_results, columns=['Job ID','Title', 'Date', 'Type', 'Description',
                                                          'Deadline', 'Salary', 'Company','Qualifications', 'Requirements'])
            with search_output:
                search_output.clear_output()
                display(df)
                back_button = widgets.Button(description='Back to Main Menu')
                back_button.on_click(lambda b: display_menu_jobseeker())
                display(back_button)
        else:
            with search_output:
                search_output.clear_output()
                print("No results found")
                back_button = widgets.Button(description='Back to Main Menu')
                back_button.on_click(lambda b: display_menu_jobseeker())
                display(back_button)

    search_button = widgets.Button(description='Search')
    search_button.on_click(search_job_callback)
    display(search_button)

# Menampilkan hasil lamaran
def display_results(b):
    clear_output()
    result_widget = widgets.Output()
    query = "SELECT jobseeker_id FROM jobseeker WHERE account_id =?"
    cursor.execute(query, (account_id,))
    jobseeker_id = cursor.fetchone()[0]
    query1 = "SELECT j.title, r.status FROM result r JOIN job j ON r.job_id = j.job_id WHERE r.jobseeker_id =?"
    cursor.execute(query1, (jobseeker_id,))
    results = cursor.fetchall()
    if not results:
        with result_widget:
            print("There is no result")
            back_button = widgets.Button(description='Back to Main Menu')
            back_button.on_click(lambda b: display_menu_jobseeker())
            display(back_button)
    else:
        df = pd.DataFrame(results, columns=["Job Title", "Status"])
        with result_widget:
            display(df)
            back_button = widgets.Button(description='Back to Main Menu')
            back_button.on_click(lambda b: display_menu_jobseeker())
            display(back_button)
    display(result_widget)

# Menampilkan jadwal interview
def display_interview(b):
    clear_output()
    interview_widget = widgets.Output()
    query = """
        SELECT j.title, c.name_com, i.date, i.time
        FROM interview i
        JOIN job j ON i.job_id = j.job_id
        JOIN company c ON j.company_id = c.company_id
        JOIN jobseeker js ON i.jobseeker_id = js.jobseeker_id
        WHERE js.account_id = ?
        """
    cursor.execute(query, (account_id,))

    results = cursor.fetchall()
    if not results:
        with interview_widget:
            print("There is no result")
            back_button = widgets.Button(description='Back to Main Menu')
            back_button.on_click(lambda b: display_menu_jobseeker())
            display(back_button)
    else:
        df = pd.DataFrame(results, columns=['Title', 'Company', 'Date', 'Time'])
        with interview_widget:
            display(df)
            back_button = widgets.Button(description='Back to Main Menu')
            back_button.on_click(lambda b: display_menu_jobseeker())
            display(back_button)
    display(interview_widget)

def apply_job_callback(b):
    job_id = job_id_input.value
    if job_id:
        cursor.execute("SELECT * FROM jobseeker WHERE account_id =?", (account_id,))
        jobseeker_data = cursor.fetchone()

        # Cek apakah job_id dan jobseeker_id sudah ada di br_apply
        cursor.execute("SELECT * FROM br_apply WHERE job_id =? AND jobseeker_id =?", (job_id, jobseeker_data[0]))
        if cursor.fetchone():
            # Jika sudah ada, tampilkan konfirmasi
            cursor.execute("SELECT j.title, c.name_com FROM job j JOIN company c ON j.company_id = c.company_id WHERE j.job_id =?", (job_id,))
            result = cursor.fetchone()
            title = result[0]
            name_com = result[1]
            print(f"You have already applied to be {title} in {name_com}")
        else:
            # Jika belum ada, insert data ke br_apply
            cursor.execute("INSERT INTO br_apply (job_id, jobseeker_id) VALUES (?,?)", (job_id, jobseeker_data[0]))
            conn.commit()

            # Mengambil job title dari tabel job dan name_com dari tabel company
            cursor.execute("SELECT j.title, c.name_com FROM job j JOIN company c ON j.company_id = c.company_id WHERE j.job_id =?", (job_id,))
            result = cursor.fetchone()
            title = result[0]
            name_com = result[1]

            print(f"You have successfully applied to be {title} in {name_com}") # Print statement berhasil apply
    else:
        print("Please insert the Job ID!")

# Melamar pekerjaan
def apply_job(b):
    clear_output()
    global job_id_input
    job_id_input = widgets.Text(value='', placeholder='Insert job_id', description='Job ID:', disabled=False)
    apply_button = widgets.Button(description='Apply')
    apply_button.on_click(apply_job_callback)
    display(job_id_input)
    display(apply_button)
    back_button = widgets.Button(description='Back to Main Menu')
    back_button.on_click(lambda b: display_menu_jobseeker())
    display(back_button)

# Menampilkan profile
def display_jobseeker_info(account_id):
    clear_output()

    cursor.execute("SELECT * FROM jobseeker WHERE account_id =?", (account_id,)) # Mengambil data jobseeker berdasarkan account_id
    jobseeker_data = cursor.fetchone()

    cursor.execute("SELECT COUNT(*) FROM br_apply WHERE jobseeker_id =?", (jobseeker_data[0],)) # Jumlah pekerjaan yang sudah dilamar
    num_jobs_applied = cursor.fetchone()[0]

    cursor.execute("SELECT link_file FROM resume WHERE jobseeker_id =?", (jobseeker_data[0],))  # Mengambil link_file dari tabel resume
    resume_data = cursor.fetchone()
    link_file = resume_data[0] if resume_data else None

    df = pd.DataFrame([{    # Buat Dataframe
        'Name': jobseeker_data[1],
        'Phone Number': jobseeker_data[2],
        'Address': jobseeker_data[3],
        'Education': jobseeker_data[4],
        'Birth Date': jobseeker_data[6],
        'Number of Jobs Applied': num_jobs_applied,
        'Resume': link_file,
    }], columns=['Name', 'Phone Number', 'Address', 'Education', 'Birth Date', 'Number of Jobs Applied', 'Resume'])

    # display
    display(df)
    update_button = widgets.Button(description='Update Profile')
    update_button.on_click(lambda b: update_profile(account_id))
    display(update_button)
    back_button = widgets.Button(description='Back to Main Menu')
    back_button.on_click(lambda b: display_menu_jobseeker())
    display(back_button)

def update_profile(account_id): # Mengupdate profile
    clear_output()

    cursor.execute("SELECT * FROM jobseeker WHERE account_id =?", (account_id,)) # Mengambil data jobseeker berdasarkan account_id
    jobseeker_data = cursor.fetchone()

    form_widgets = [ #Form untuk input
        widgets.Text(value=jobseeker_data[1], placeholder='Name', description='Name:', style={'description_width': '100px'},
                     layout=Layout(width='300px', align_items='flex-start')),
        widgets.Text(value=jobseeker_data[2], placeholder='Phone Number', description='Phone Number:', style={'description_width': '100px'},
                     layout=Layout(width='300px', align_items='flex-start')),
        widgets.Textarea(value=jobseeker_data[3], placeholder='Address', description='Address:', style={'description_width': '100px'},
                         layout=Layout(width='300px', align_items='flex-start')),
        widgets.Dropdown(options=['High School','Bachelor\'s Degree', 'Master\'s Degree', 'PhD'],
                        value=jobseeker_data[4],
                        description='Education:',
                        style={'description_width': '100px'},
                        layout=Layout(width='300px', align_items='flex-start')),
        widgets.DatePicker(value=datetime.strptime(jobseeker_data[6], '%d/%m/%Y').date(), description='Birth Date:', style={'description_width': '100px'},
                           layout=Layout(width='300px', align_items='flex-start'))
    ]

    def update_profile_callback(b): # Fungsi untuk update profile
        name = form_widgets[0].value
        phone_number = form_widgets[1].value
        address = form_widgets[2].value
        education = form_widgets[3].value
        birth_date = form_widgets[4].value

        # Update data jobseeker
        cursor.execute("UPDATE jobseeker SET name_js=?, phone_number=?, address=?, education=?, birth_date=? WHERE account_id=?", (name, phone_number, address, education, birth_date, account_id))
        conn.commit()

        print("Profile updated successfully!") # Print statement setelah berhasil
        back_button = widgets.Button(description='Back to Main Menu')
        back_button.on_click(lambda b: display_menu_jobseeker())
        display(back_button)

    update_button = widgets.Button(description='Submit')
    update_button.on_click(update_profile_callback)

    # display
    display(*form_widgets)
    display(update_button)

def update_resume(b): # Mengupdate resume
    clear_output()

    link_input = widgets.Text(value='', placeholder='Enter the link to your resume', description='Link:', disabled=False)
    display(link_input)

    def upload_resume(b):
        cursor.execute("SELECT jobseeker_id FROM jobseeker WHERE account_id =?", (account_id,))
        jobseeker_id = cursor.fetchone()[0]

        cursor.execute("SELECT * FROM resume WHERE jobseeker_id =?", (jobseeker_id,)) # Periksa apakah jobseeker_id sudah ada di tabel resume
        if cursor.fetchone():
            cursor.execute("UPDATE resume SET link_file=? WHERE jobseeker_id=?", (link_input.value, jobseeker_id)) # Jika ya, update
            conn.commit()
            print("Resume updated successfully!")
        else:
            cursor.execute("INSERT INTO resume (jobseeker_id, link_file) VALUES (?,?)", (jobseeker_id, link_input.value)) # Jika tidak, masukkan record baru
            conn.commit()
            print("Resume added successfully!")

        back_button = widgets.Button(description='Back to Main Menu')
        back_button.on_click(lambda b: display_menu_jobseeker())
        display(back_button)

    # display
    upload_button = widgets.Button(description='Submit')
    upload_button.on_click(upload_resume)
    display(upload_button)

## Menu Company
def display_menu_company():
    clear_output()
    print("Main Menu")
    company_menu_widget = widgets.VBox([
        widgets.Button(description='Post Job', button_style='info', tooltip='Post a new job'),
        widgets.Button(description='Jobseeker', button_style='info', tooltip='View applicants for your jobs'),
        widgets.Button(description='Interview', button_style='info', tooltip='Manage interviews'),
        widgets.Button(description='Result', button_style='info', tooltip='Add result for the jobseeker')
    ])
    display(company_menu_widget)
    post_job_button = company_menu_widget.children[0]
    post_job_button.on_click(lambda b: post_job(b))
    view_applicants_button = company_menu_widget.children[1]
    view_applicants_button.on_click(lambda b: view_applicants(account_id))
    interview_button = company_menu_widget.children[2]
    interview_button.on_click(lambda b: display_interview_menu())
    add_result_button = company_menu_widget.children[3]
    add_result_button.on_click(lambda b: create_interview_result(account_id))

# Menu Interview
def display_interview_menu():
    clear_output()
    interview_menu_widget = widgets.VBox([
        widgets.Button(description='View Interview', button_style='info', tooltip='View scheduled interviews'),
        widgets.Button(description='Create New Schedule', button_style='info', tooltip='Create a new interview schedule')
    ])
    display(interview_menu_widget)
    view_schedule_button = interview_menu_widget.children[0]
    view_schedule_button.on_click(lambda b: view_schedule(b))
    create_schedule_button = interview_menu_widget.children[1]
    create_schedule_button.on_click(lambda b: create_interview_schedule(account_id))
    back_button = widgets.Button(description='Back to Main Menu')
    back_button.on_click(lambda b: display_menu_company())
    display(back_button)

# Menampilkan jadwal interview
def view_schedule(b):
    clear_output()
    output = widgets.Output()

    # Mengambil data jobseeker yang sudah melamar di company
    query = """
       SELECT js.jobseeker_id, js.name_js, j.title, i.date, i.time
       FROM jobseeker js
       JOIN interview i ON js.jobseeker_id = i.jobseeker_id
       JOIN job j ON i.job_id = j.job_id
       WHERE j.company_id = (SELECT c.company_id FROM company c WHERE c.account_id =?)
    """
    cursor.execute(query, (account_id,)) # Berdasarkan account_id
    results = cursor.fetchall()

    if not results: # Jika tidak ada, print
        output.clear_output()
        with output:
             print("There are no interview scheduled.")
    else:
       df = pd.DataFrame(results, columns=['Jobseeker ID', 'Name', 'Job Title', 'Date','Time']) # Buat Dataframe

       with output:
           display(df)

    # display
    display(output)
    back_button = widgets.Button(description='Back to Main Menu')
    back_button.on_click(lambda b: display_menu_company())
    display(back_button)

# Membuat jadwal interview
def create_interview_schedule(account_id):
    clear_output()

    cursor.execute("SELECT company_id FROM company WHERE account_id =?", (account_id,)) # Mengambil company_id berdasar account_id
    company_id = cursor.fetchone()[0]

    cursor.execute("SELECT job_id, title FROM job WHERE company_id =?", (company_id,)) # Mengambil job_id berdasar account_id
    job_ids = cursor.fetchall()

    # input
    job_id_input = widgets.Dropdown(options=[(job_id, title) for job_id, title in job_ids], description='Job ID:', disabled=False)
    date_input = widgets.DatePicker(description='Date:', disabled=False)
    time_input = widgets.Text(value='', placeholder='HH:MM', description='Time:', disabled=False)
    jobseeker_id_input = widgets.Text(value='', placeholder='Jobseeker ID', description='Jobseeker ID:', disabled=False)

    create_button = widgets.Button(description="Create Schedule", button_style='success') # tombol untuk create schedule

    def create_schedule(b):
        job_title = job_id_input.value
        date = date_input.value
        if date is not None:
            date_str = date.strftime('%d/%m/%Y')
        else:
            print("Please select a date!")
            return
        time = time_input.value
        jobseeker_id = jobseeker_id_input.value

        cursor.execute("SELECT job_id FROM job WHERE title =?", (job_title,)) # job_id berdasar title
        job_id_row = cursor.fetchone()
        if job_id_row:
            job_id = job_id_row[0]
        else:
            print("Job title not found!")
            return

        # Periksa apakah sudah melamar
        cursor.execute("SELECT 1 FROM br_apply WHERE job_id =? AND jobseeker_id =?", (job_id, jobseeker_id))
        if cursor.fetchone() is not None:
            # Periksa apakah sudah ada jadwal
            cursor.execute("SELECT 1 FROM interview WHERE job_id =? AND jobseeker_id =?", (job_id, jobseeker_id))
            if cursor.fetchone() is not None:
                print(f"The job applicant has an interview schedule for the job with job_id {job_id}. Please input it correctly.")
                return
            else:
                cursor.execute("INSERT INTO interview (job_id, jobseeker_id, date, time) VALUES (?,?,?,?)", (job_id, jobseeker_id, date_str, time))
                conn.commit()
                print("The interview schedule has been created!")
        else:
            print("Jobseeker has not yet applied for the job!")

    create_button.on_click(create_schedule)

    # display interface
    display(job_id_input)
    display(date_input)
    display(time_input)
    display(jobseeker_id_input)
    display(create_button)
    back_button = widgets.Button(description='Back to Main Menu')
    back_button.on_click(lambda b: display_menu_company())
    display(back_button)

# Menampilkan pelamar kerja
def view_applicants(b):
    clear_output()
    output = widgets.Output()

    # Mengambil data jobseeker yang sudah melamar di company
    query = """
       SELECT js.jobseeker_id, js.name_js, js.phone_number, js.address, js.education, r.link_file, j.title, js.birth_date
       FROM jobseeker js
       JOIN resume r ON js.jobseeker_id = r.jobseeker_id
       JOIN br_apply ap ON js.jobseeker_id = ap.jobseeker_id
       JOIN job j ON ap.job_id = j.job_id
       WHERE j.company_id = (SELECT c.company_id FROM company c WHERE c.account_id =?)
    """

    cursor.execute(query, (account_id,)) # berdasar account_id
    results = cursor.fetchall()

    if not results:
        output.clear_output()
        with output:
             print("There are no jobseekers who apply at this company.") # apabila tidak ada data
    else:
       # buat dataframe
       df = pd.DataFrame(results, columns=['Jobseeker ID', 'Name', 'Phone Number', 'Address', 'Education', 'Resume', 'Job Title', 'Birth Date'])

       df['Age'] = df['Birth Date'].apply(lambda x: calculate_age(x)) # menambahkan kolom age

       df = df.drop('Birth Date', axis=1) # hapus kolom birth date

       # display
       output.clear_output()
       with output:
            display(df)

    display(output)   # display interface
    back_button = widgets.Button(description='Back to Main Menu')
    back_button.on_click(lambda b: display_menu_company())
    display(back_button)

def calculate_age(birth_date): # menghitung umur
    today = datetime.today()
    birth_date = datetime.strptime(birth_date, '%d/%m/%Y')
    return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))

# Menambahkan job/lamaran kerja baru
def post_job(b):
    clear_output()

    form_widgets = [
        widgets.Text(value='', placeholder='Title', description='Title:'),
        widgets.DatePicker(value=date.today(), description='Today:'),
        widgets.Dropdown(options=['Part-time', 'Full-time', 'Freelance'], description='Type:'),
        widgets.Textarea(value='', placeholder='Description', description='Description:'),
        widgets.DatePicker(value=date.today(), description='Deadline:'),
        widgets.Text(value='', placeholder='Range Salary (xxx-xxx)', description='Salary:'),
        widgets.Textarea(value='', placeholder='Requirement (Setiap kalimat dipisahkan dengan ;)', description='Requirement:'),
        widgets.Textarea(value='', placeholder='Qualification (Setiap kalimat dipisahkan dengan ;)', description='Qualification:')
    ]

    submit_button = Button(description="Submit", button_style='info')
    submit_button.on_click(lambda x: add_job_to_db(account_id, *[widget.value for widget in form_widgets]))

    # display
    display(*form_widgets)
    display(submit_button)
    back_button = widgets.Button(description='Back to Main Menu')
    back_button.on_click(lambda b: display_menu_company())
    display(back_button)

def add_job_to_db(account_id, title, date, type_job, description, deadline, salary, requirement, qualification):

    cursor.execute("SELECT company_id FROM company WHERE account_id =?", (account_id,)) # Mengambil data berdasar account_id
    company_id = cursor.fetchone()[0]

    # Insert job baru
    cursor.execute("INSERT INTO job (title, date, type, description, deadline, salary, company_id) VALUES (?,?,?,?,?,?,?)", (str(title), date, type_job, description, deadline, salary, company_id))
    conn.commit()

    cursor.execute("SELECT last_insert_rowid()")
    job_id = cursor.fetchone()[0]

    # Memisahkan antara kalimat masing-masing requirement dan qualification dengan ;
    requirements = requirement.split(';')
    qualifications = qualification.split(';')

    for req in requirements:
        cursor.execute("SELECT req_id FROM requir WHERE requirement =?", (req,))
        req_id = cursor.fetchone()
        if req_id is not None:
           req_id = req_id[0]
           cursor.execute("INSERT INTO br_requir (req_id, job_id) VALUES (?,?)", (req_id, job_id)) # Insert data ke br_requir
           conn.commit()
        else:
           cursor.execute("INSERT INTO requir (requirement) VALUES (?)", (req,))
           conn.commit()
           cursor.execute("SELECT last_insert_rowid()")
           req_id = cursor.fetchone()[0]
           cursor.execute("INSERT INTO br_requir (req_id, job_id) VALUES (?,?)", (req_id, job_id)) # Insert data ke br_requir
           conn.commit()

    for qual in qualifications:
        cursor.execute("SELECT qualif_id FROM qualif WHERE qualification =?", (qual,))
        qual_id = cursor.fetchone()
        if qual_id is not None:
           qual_id = qual_id[0]
           cursor.execute("INSERT INTO br_requir (req_id, job_id) VALUES (?,?)", (qual_id, job_id)) # Insert data ke br_qualif
           conn.commit()
        else:
           cursor.execute("INSERT INTO qualif (qualification) VALUES (?)", (qual,))
           conn.commit()
           cursor.execute("SELECT last_insert_rowid()")
           qual_id = cursor.fetchone()[0]
           cursor.execute("INSERT INTO br_requir (req_id, job_id) VALUES (?,?)", (qual_id, job_id)) # Insert data ke br_qualif
           conn.commit()

    print("Job has been added!") # print statement setelah berhasil

# Menambahkan hasil interview
def create_interview_result(account_id):
    clear_output()

    cursor.execute("SELECT company_id FROM company WHERE account_id =?", (account_id,)) # company_id berdasar account_id
    company_id = cursor.fetchone()[0]

    cursor.execute("SELECT job_id, title FROM job WHERE company_id =?", (company_id,)) # job_id berdasar company_id
    job_ids = cursor.fetchall()

    cursor.execute("SELECT jobseeker_id FROM jobseeker WHERE account_id =?", (account_id,))
    jobseeker_ids = cursor.fetchall()

    # menu untuk input
    job_id_input = widgets.Dropdown(options=[(job_id, title) for job_id, title in job_ids], description='Job ID:', disabled=False)
    jobseeker_id_input = widgets.Text(value='', placeholder='Jobseeker ID', description='Jobseeker ID:', disabled=False)
    status_input = widgets.Dropdown(options=[('Accepted', 'Accepted'), ('Not Accepted', 'Not Accepted')], description='Status:', disabled=False)

    add_button = widgets.Button(description="Add Result", button_style='success') # tombol add result

    def add_result(b):
        job_id = job_id_input.value
        jobseeker_id = jobseeker_id_input.value
        status = status_input.value

        cursor.execute("SELECT 1 FROM br_apply WHERE job_id =? AND jobseeker_id =?", (job_id, jobseeker_id))
        if cursor.fetchone() is not None:
            cursor.execute("SELECT 1 FROM result WHERE job_id =? AND jobseeker_id =?", (job_id, jobseeker_id))
            if cursor.fetchone() is not None:
                cursor.execute("UPDATE result SET status =? WHERE job_id =? AND jobseeker_id =?", (status, job_id, jobseeker_id))
            else:
                cursor.execute("INSERT INTO result (job_id, jobseeker_id, status) VALUES (?,?,?)", (job_id, jobseeker_id, status))
            conn.commit()
            print("The result has been added!")
        else:
            print("Jobseeker has not yet applied for the job!")

    add_button.on_click(add_result)

    # display interface
    display(job_id_input)
    display(jobseeker_id_input)
    display(status_input)
    display(add_button)
    back_button = widgets.Button(description='Back to Main Menu')
    back_button.on_click(lambda b: display_menu_company())
    display(back_button)

login_button.on_click(login)
register_button.on_click(register)

display(email_widget, password_widget, login_button, register_button, output_widget)

connect_to_db()

Text(value='', description='Email:', style=DescriptionStyle(description_width='initial'))

Password(description='Password:', style=DescriptionStyle(description_width='initial'))

Button(button_style='success', description='Login', style=ButtonStyle(), tooltip='Login to your account')

Button(button_style='info', description='Register', style=ButtonStyle(), tooltip='Create a new account')

Output()

# Penjelasan

Fungsi:

1.   **connect_to_db()**: Fungsi ini digunakan untuk membuat koneksi ke database.
2.   **enter(email, password)**: Fungsi ini digunakan untuk memeriksa apakah email yang diinput sudah pernah digunakan sebelumnya. Jika email belum pernah digunakan, maka fungsi ini akan menambahkan data ke dalam tabel account.
3. **check(email, password)**: Fungsi ini digunakan untuk memeriksa apakah email dan password yang diinput sesuai dengan data yang ada di tabel account.
4. **login(b)**: Fungsi ini digunakan untuk memeriksa apakah email dan password yang diinput sesuai dengan data yang ada di tabel account. Jika sesuai, maka fungsi ini akan menampilkan menu sesuai dengan jenis akun yang diinput (jobseeker atau company).
5. **register(b)**: Fungsi ini digunakan untuk memeriksa apakah email yang diinput sudah pernah digunakan sebelumnya. Jika email belum pernah digunakan, maka fungsi ini akan menambahkan data ke dalam tabel account dan menampilkan menu sesuai dengan jenis akun yang diinput (jobseeker atau company).

display_menu_jobseeker(): Fungsi ini digunakan untuk menampilkan menu utama bagi jobseeker.
1. **search_job(b)**: Fungsi ini digunakan untuk mencari lowongan kerja sesuai dengan judul yang diinput.
2. **display_results(b)**: Fungsi ini digunakan untuk menampilkan hasil pencarian lowongan kerja.
3. **display_interview(b)**: Fungsi ini digunakan untuk menampilkan jadwal interview yang sudah disetujui.
4. **apply_job_callback(b)**: Fungsi ini digunakan untuk mengirim permohonan lowongan kerja.
5. **display_jobseeker_info(account_id)**: Fungsi ini digunakan untuk menampilkan informasi jobseeker.
6. **update_profile(account_id)**: Fungsi ini digunakan untuk memperbarui profil jobseeker.
7. **update_resume(b)**: Fungsi ini digunakan untuk memperbarui resume jobseeker.

display_menu_company(): Fungsi ini digunakan untuk menampilkan menu utama bagi company.
1. **display_company_info(account_id)**: Fungsi ini digunakan untuk menampilkan informasi company.
2. **post_job(b)**: Fungsi ini digunakan untuk membuat lowongan kerja baru.
view_applicants(b): Fungsi ini digunakan untuk menampilkan daftar jobseeker yang mengajukan lowongan kerja.
3. **interview_schedule(b)**: Fungsi ini digunakan untuk mengatur jadwal interview.
4. **create_interview_result(account_id)**: Fungsi ini digunakan untuk menambahkan hasil interview ke dalam tabel result.